In [14]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from sklearn import metrics 

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic",
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


<img src='./data/Method_allfit_Sequential.jpg' width=600px>

# Training - all fit

In [40]:
import random    
seed_num = 42
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

In [41]:
def get_model(gpu_mode=False):
    seed_num = 42
    tf.random.set_seed(seed_num)
    if gpu_mode:
        lstm = Sequential()
        lstm.add(InputLayer(input_shape=(X.shape[1],X.shape[2])))
        lstm.add(CuDNNLSTM(units=128, return_sequences=True))
        lstm.add(Activation('hard_sigmoid'))
        lstm.add(CuDNNLSTM(units=64, return_sequences=True))
        lstm.add(Activation('hard_sigmoid'))
        lstm.add(Dropout(0.2))
        lstm.add(CuDNNLSTM(units=64, return_sequences=True))
        lstm.add(Activation('hard_sigmoid'))
        lstm.add(CuDNNLSTM(units=32, return_sequences=False))
        lstm.add(Activation('hard_sigmoid'))
        lstm.add(Dropout(0.2))
        lstm.add(Dense(units=1, activation='sigmoid'))
    else:
        lstm = Sequential()
        lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
        lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
        lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
        lstm.add(Dropout(0.2))
        lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
        lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
        lstm.add(Dropout(0.2))
        lstm.add(Dense(units=1, activation='sigmoid'))

    optimizer = Adam(learning_rate = 0.001)
    lstm.compile(optimizer=optimizer, loss = "binary_crossentropy", metrics=['acc'])
    return lstm

In [43]:
MODEL_SAVE_FOLDER_PATH = './models/'
filepath = MODEL_SAVE_FOLDER_PATH + 'ALLFIT1_{epoch:02d}-{val_acc:.4f}.hdf5'
ckpt = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, save_weights_only=False, save_freq='epoch')

model = get_model(gpu_mode=False)
early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
model.fit(X, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop, ckpt], shuffle=False)

Epoch 1/500
46/46 [==============================] - 7s 82ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865
Epoch 2/500
46/46 [==============================] - 3s 69ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865
Epoch 3/500
46/46 [==============================] - 3s 67ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865
Epoch 4/500
46/46 [==============================] - 3s 69ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865
Epoch 5/500
46/46 [==============================] - 3s 70ms/step - loss: 0.5325 - acc: 0.7370 - val_loss: 0.5727 - val_acc: 0.7091
Epoch 6/500
46/46 [==============================] - 3s 68ms/step - loss: 0.4406 - acc: 0.8109 - val_loss: 0.5526 - val_acc: 0.7340
Epoch 7/500
46/46 [==============================] - 3s 69ms/step - loss: 0.3932 - acc: 0.8412 - val_loss: 0.5593 - val_acc: 0.7407
Epoch 8/500
46/46 [==============================] - 3s 67ms/step - loss: 0.

Epoch 58/500
46/46 [==============================] - 3s 68ms/step - loss: 0.0799 - acc: 0.9803 - val_loss: 1.1117 - val_acc: 0.7257
Epoch 59/500
46/46 [==============================] - 3s 68ms/step - loss: 0.0901 - acc: 0.9765 - val_loss: 1.0791 - val_acc: 0.7220
Epoch 60/500
46/46 [==============================] - 3s 69ms/step - loss: 0.0826 - acc: 0.9786 - val_loss: 1.0558 - val_acc: 0.7272
Restoring model weights from the end of the best epoch.
Epoch 00060: early stopping


In [ ]:
# model = tf.keras.models.load_model('./models/ALLFIT_17-0.7645.hdf5')
# model = tf.keras.models.load_model('./models/ALLFIT_01-0.4865.hdf5')

## Check accuracy

In [45]:
model = tf.keras.models.load_model('./models/ALLFIT1_10-0.7578.hdf5')

dic={}
for seed in range(0, 50, 5):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7980582524271844, seed_num = 0
정확도 :0.7974110032362459, seed_num = 5
정확도 :0.8077669902912621, seed_num = 10
정확도 :0.8032362459546926, seed_num = 15
정확도 :0.7896440129449838, seed_num = 20
정확도 :0.8122977346278317, seed_num = 25
정확도 :0.8181229773462784, seed_num = 30
정확도 :0.7870550161812297, seed_num = 35
정확도 :0.7902912621359224, seed_num = 40
정확도 :0.7915857605177994, seed_num = 45
seed = 42의 정확도 df 만들고 평균 확인 : [0.79954693]


# Entropy dict

In [20]:
COLS = list(pd.read_csv('/project/LSH/total_data_7727.csv')['ITEMID'].sort_values().unique())

In [21]:
def entropy(ratio_list):
    one_ratio, zero_ratio = ratio_list[0], ratio_list[1] 
    return - ((one_ratio * (np.log2(one_ratio))) + (zero_ratio * (np.log2(zero_ratio))))

X = np.load('/project/LSH/x_(7727,10,4068).npy')

entropy_dict = {}
for i in tqdm(range(len(COLS))):
    one_ratio = X[:,:,i].sum() / (X.shape[0]*X.shape[1])
    zero_ratio = 1 - one_ratio
    entropy_num = entropy([one_ratio, zero_ratio])
    entropy_dict[COLS[i]] = entropy_num

  0%|          | 0/4068 [00:00<?, ?it/s]

In [22]:
entropy_dict

{0: 0.8945475095722644,
 50803: 0.033442542699476635,
 50804: 0.48487374976369707,
 50805: 0.0011721468075888046,
 50806: 0.07466452012902748,
 50808: 0.3422567690915075,
 50809: 0.30631983549914066,
 50811: 0.17607571914919884,
 50813: 0.3020503924033536,
 50814: 0.0013473569150405915,
 50818: 0.5905150583063137,
 50820: 0.5572216411567222,
 50821: 0.6928844725130754,
 50822: 0.14596543633389195,
 50824: 0.075539219929809,
 50852: 0.028723919067390347,
 50853: 0.0018581977954349603,
 50856: 0.007513335654158085,
 50858: nan,
 50861: 0.4000048713984587,
 50862: 0.4705525702453669,
 50863: 0.4083236220110865,
 50864: 0.0011721468075888046,
 50865: 0.0025135191238101528,
 50866: 0.02373004513245036,
 50867: 0.11824848231980385,
 50868: 0.2937998951444971,
 50871: nan,
 50872: nan,
 50873: 0.0006248993908785407,
 50876: 0.00022881204196882252,
 50877: 0.00022881204196882252,
 50878: 0.45076509428156675,
 50881: 0.0018581977954349603,
 50882: 0.7677642918971788,
 50883: 0.07702015471713725

# 2. {E(0to1) - E(1to0)} * Entropy ^ Lambda (0 or 1)

In [15]:
with tf.device('/device:GPU:1'): 
    model = tf.keras.models.load_model('./models/ALLFIT_17-0.7645.hdf5')
    result = []
    for i in tqdm(range(X.shape[2])):
        save_cols = X[:,:,i].copy()
        #-----zero2one-----
        X[:,:,i] = 1
        pred1 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
        mean_pred1 = np.mean(pred1)
        #-----one2zero-----
        X[:,:,i] = 0
        pred2 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
        mean_pred2 = np.mean(pred2)

        result.append({'feature' : str(COLS[i]), 'zero2one' : mean_pred1, 'one2zero' : mean_pred2,
                       'lambda0' : mean_pred1 - mean_pred2, 'lambda1' : (mean_pred1 - mean_pred2) * entropy_dict[COLS[i]]})

        #값 복원
        X[:,:,i] = save_cols
        break

  0%|          | 0/4068 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame(result).sort_values('lambda0', ascending=False)
df

,feature,one2zero,zero2one,lambda0,lambda1
0,0,0.789104,0.754038,0.035065,0.031368


# 3. Feature Importance - Sequential Method

In [6]:
import json

#-----FI 측정 시작-----
with tf.device('/device:GPU:0'): 
    #-----모델 로드-----
    model = tf.keras.models.load_model('./models/ALLFIT_17-0.7645.hdf5')
    top10 = []
    top10_result = []
    for n in tqdm(range(1), desc="Top10 Iteration"):
        result = []
        if len(top10) > 1:
            print('####################  top10 :', [COLS[i] for i in top10])
            for i in tqdm(range(X.shape[2]), desc="zero, one Predict"):
                save_cols = X[:,:,top10].copy()
                #-----zero2one-----
                X[:,:,top10] = 1
                pred1 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
                mean_pred1 = np.mean(pred1)
                #-----one2zero-----
                X[:,:,top10] = 0
                pred2 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
                mean_pred2 = np.mean(pred2)
                result.append({'feature_index' : i
                            'lambda0' : mean_pred1 - mean_pred2, 'lambda1' : (mean_pred1 - mean_pred2) * entropy_dict[COLS[i]]})
            df = pd.DataFrame(result).sort_values('lambda0', ascending=False)
            print(df)
            top10.append(df.feature_index[n])
        else:
            for i in tqdm(range(X.shape[2]), desc="zero, one Predict"):
                save_cols = X[:,:,i].copy()
                #-----zero2one-----
                X[:,:,i] = 1
                pred1 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
                mean_pred1 = np.mean(pred1)
                #-----one2zero-----
                X[:,:,i] = 0
                pred2 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
                mean_pred2 = np.mean(pred2)

                result.append({'feature_index' : i, 'zero2one' : mean_pred1, 'one2zero' : mean_pred2,
                            'lambda0' : mean_pred1 - mean_pred2, 'lambda1' : (mean_pred1 - mean_pred2) * entropy_dict[COLS[i]]})

            df = pd.DataFrame(result).sort_values('lambda0', ascending=False)
            top10.append(df.feature_index[0])


#     #-----df저장-----
#     df.to_csv('./data/Method_Sequential_FI.csv', index=False)
#     #-----top10 feature만 저장-----
#     top10_list = list(map(lambda x:str(COLS[x]), top10))
#     print(top10_list)
#     with open('./data/Sequential_top10.txt', "w", encoding="UTF-8") as file:
#         json.dump(top10_list, file, ensure_ascii=False)

Top10 Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

zero, one Predict:   0%|          | 0/4068 [00:00<?, ?it/s]

zero, one Predict:   0%|          | 0/4068 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
df

,feature_index,one2zero,zero2one,lambda0,lambda1
2611,2611,0.507915,0.507874,0.000041,0.000023
3832,3832,0.507955,0.507916,0.000040,0.000032
2388,2388,0.507905,0.507868,0.000037,0.000022
3456,3456,0.507941,0.507908,0.000034,0.000018
619,619,0.507929,0.507899,0.000030,0.000018
...,...,...,...,...,...
12,12,0.508100,0.508144,-0.000043,-0.000030
10,10,0.508113,0.508157,-0.000044,-0.000026
11,11,0.508107,0.508152,-0.000045,-0.000025
56,56,0.508046,0.508094,-0.000048,-0.000041


## Check

In [33]:
import pandas as pd
try1 = pd.read_csv('./data/Method_eel_FI.csv').sort_values('feature').reset_index(drop=True)
try1['lambda0'] = try1['zero2one'] - try1['one2zero'] 
try1['lambda1'] = try1['lambda0'] * list(entropy_dict.values())

In [36]:
try1.sort_values('lambda0',ascending=False)

,feature,one2zero,zero2one,lambda0,lambda1
3789,62856024641,0.345287,0.905962,0.560675,0.006665
3982,67159011203,0.382268,0.931299,0.549031,0.004575
4020,68084024711,0.376804,0.923922,0.547118,0.011353
3987,67286081203,0.380851,0.921074,0.540222,0.014689
3586,59011010325,0.376409,0.915132,0.538723,0.004490
...,...,...,...,...,...
3836,63323026965,0.446835,0.130318,-0.316518,-0.069818
3882,63459010001,0.425043,0.104555,-0.320488,-0.000376
2648,904564061,0.463617,0.138598,-0.325019,-0.001322
3871,63323065102,0.428467,0.096872,-0.331595,-0.001789


In [9]:
try2 = pd.read_csv('./data/Method_eel_FI_3852.csv').sort_values('lambda0', ascending=False)
try2[try2['feature']==63323038810]

,feature,one2zero,zero2one,lambda0,lambda1
3972,51277,0.51719,0.347983,0.169207,0.155557


In [13]:
try2.sort_values('one2zero')

,feature,one2zero,zero2one,lambda0,lambda1
3341,63323011710,0.344205,0.070149,0.274056,0.001073
3163,63323012250,0.344214,0.061326,0.282888,0.000230
3703,63323010510,0.344280,0.106804,0.237477,0.011034
3203,63323014210,0.344323,0.063023,0.281300,0.000428
3373,63323013210,0.344324,0.072559,0.271765,0.000319
...,...,...,...,...,...
3954,50866,0.655095,0.478588,0.176506,0.004189
1085,50867,0.656576,0.307940,0.348636,0.041226
3278,50878,0.658026,0.381286,0.276740,0.124745
3514,50881,0.658030,0.398346,0.259684,0.000483


In [8]:
COLS.index(50856)

17

# Visualization

In [ ]:
def visualization(top10_list):
    #-----데이터 로드-----
    X = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')
    #-----컬럼이름 로드-----
    COLS = list(pd.read_csv('/project/LSH/total_data_7727.csv')['ITEMID'].sort_values().unique())
    #-----사망/퇴원 환자 분리-----
    d_index, s_index = np.where(y==1)[0], np.where(y==0)[0]
    d_X, s_X = X[d_index], X[s_index]
    result_d, result_s = [], []
    #-----사망/퇴원 환자별 item 1의 합계 구하기-----
    #day - 10일
    for d in range(10):
        #4068 - ITEM
        for f in range(d_X.shape[-1]):
            d_sum = d_X[:,d,f].sum()/d_X.shape[0]
            s_sum = s_X[:,d,f].sum()/s_X.shape[0]
            result_d.append({'cols':COLS[f], 'day':10-d,'per':d_sum})
            result_s.append({'cols':COLS[f], 'day':10-d,'per':s_sum})
    #-----최종 합계 df-----
    d_df = pd.DataFrame(result_d).sort_values(['cols','day']).reset_index(drop=True)
    s_df = pd.DataFrame(result_s).sort_values(['cols','day']).reset_index(drop=True)
    #-----Visualization-----
    plt.figure(figsize = (13,12), dpi=150)
    for i, f in enumerate(top10_list):
        plt.subplot(4,3,1+i)
        plt.title(f)
        ax = sns.lineplot(data = d_df[d_df['cols']==int(f)], x = 'day', y='per', label='사망')
        ax = sns.lineplot(data = s_df[s_df['cols']==int(f)], x = 'day', y='per', label='퇴원')
        ax.invert_xaxis()
        ax.legend(loc='upper left')
    plt.tight_layout()